**3. Etapa de entrenamiento y testeo de un modelo de análisis
de sentimiento**


---


Importamos las librerías necesarias y cargamos el dataset

In [2]:
import pandas as pd

In [4]:
# Cargamos los datos preprocesados
df = pd.read_csv('processed_pet_supplies_reviews.csv')

# Mostramos las primeras 5 filas del DataFrame
print(df.head())


   overall  verified   reviewTime      reviewerID        asin  \
0      5.0      True  05 27, 2014   AVA8KXONIGNV4  B000FPH1CA   
1      5.0      True  02 16, 2014   A60D5HQFOTSOM  B005GSEC3W   
2      5.0      True  09 22, 2014  A1BBTVAGN6YIGD  B000WFKTWM   
3      1.0     False  08 18, 2017   AS3JM3ZLNJ5DR  B010MVG6ZY   
4      2.0      True  09 25, 2016  A38S15PFJIWRYU  B00JZIDCC6   

                                               style     reviewerName  \
0  {'Flavor Name:': ' Song Plus', 'Pattern:': ' C...  Victoria Puffer   
1  {'Size:': ' 18" L X 13" W X 13" H', 'Color:': ...   DanCooperMedia   
2                           {'Size:': ' 16 lb. Bag'}    ShootingStarz   
3  {'Size:': ' 100-Count Wipes', 'Color:': ' Unsc...        B. Graham   
4  {'Size:': ' 9-Count', 'Flavor Name:': ' Chicke...  Amazon Customer   

                                          reviewText  \
0  This product and actualy be eatten by a human....   
1  I have a Persian cat. He likes sleeping in thi...   
2 

Modificaremos el código que hemos creado en la práctica de machine learning para adaptarlo para este ejercicio 3.


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Eliminamos filas con valores NaN en 'processed_reviewText'
df = df.dropna(subset=['processed_reviewText'])

# Verificamos el tamaño del dataset después de eliminar NaN
dataset_size = len(df)
print(f"Dataset size after dropping NaN: {dataset_size}")

# Usamos una muestra del dataset si el tamaño es mayor a 50,000, de lo contrario usamos todo el dataset
sample_size = min(50000, dataset_size)
df_sample = df.sample(n=sample_size, random_state=42)

# Definimos las características y la variable objetivo
X = df_sample['processed_reviewText']
y = df_sample['overall'].apply(lambda rating: 1 if rating >= 4 else 0)  # 1 para positivo, 0 para negativo

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Creamos el vectorizador Bag-of-Words
vectorizer = CountVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 3)) # ngram_range ya que, como hemos visto, los trigramas también ofrecen información

# Ajustamos y transformamos los datos de entrenamiento
X_train_bow = vectorizer.fit_transform(X_train)

# Transformamos los datos de prueba
X_test_bow = vectorizer.transform(X_test)

# Entrenamos el modelo de Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=100)
lr_model.fit(X_train_bow, y_train)

# Predecimos los valores en el conjunto de prueba
y_pred_lr = lr_model.predict(X_test_bow)

# Entrenamos el modelo de Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=50, max_depth=10)
rf_model.fit(X_train_bow, y_train)

# Predecimos los valores en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test_bow)

# Evaluamos el modelo de Logistic Regression
print("Logistic Regression Model:")
print(classification_report(y_test, y_pred_lr))

# Evaluamos el modelo de Random Forest
print("Random Forest Model:")
print(classification_report(y_test, y_pred_rf))

# Comparamos las métricas
lr_report = classification_report(y_test, y_pred_lr, output_dict=True)
rf_report = classification_report(y_test, y_pred_rf, output_dict=True)

# Mostramos las métricas de precisión, recall y f1-score para ambos modelos
print(f"Logistic Regression - Precision: {lr_report['1']['precision']}, Recall: {lr_report['1']['recall']}, F1-score: {lr_report['1']['f1-score']}")
print(f"Random Forest - Precision: {rf_report['1']['precision']}, Recall: {rf_report['1']['recall']}, F1-score: {rf_report['1']['f1-score']}")


Dataset size after dropping NaN: 49940


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Model:
              precision    recall  f1-score   support

           0       0.70      0.44      0.54      2017
           1       0.87      0.95      0.91      7971

    accuracy                           0.85      9988
   macro avg       0.79      0.69      0.72      9988
weighted avg       0.84      0.85      0.83      9988

Random Forest Model:
              precision    recall  f1-score   support

           0       0.94      0.02      0.05      2017
           1       0.80      1.00      0.89      7971

    accuracy                           0.80      9988
   macro avg       0.87      0.51      0.47      9988
weighted avg       0.83      0.80      0.72      9988

Logistic Regression - Precision: 0.8697094486387554, Recall: 0.9538326433320787, F1-score: 0.9098306707353556
Random Forest - Precision: 0.8020130850528435, Recall: 0.9996236356793377, F1-score: 0.8899810119513012


Vamos a intentar mejorar estos primeros parámetros para ver que sucede por ahora tenemos en cuenta que el mejor de todos ha sido la logística

In [7]:
# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Cargamos los datos preprocesados
df = pd.read_csv('processed_pet_supplies_reviews.csv')

# Eliminamos filas con valores NaN en 'processed_reviewText'
df = df.dropna(subset=['processed_reviewText'])

# Verificamos el tamaño del dataset después de eliminar NaN
dataset_size = len(df)
print(f"Dataset size after dropping NaN: {dataset_size}")

# Usamos una muestra del dataset si el tamaño es mayor a 50,000, de lo contrario usamos todo el dataset
sample_size = min(50000, dataset_size)
df_sample = df.sample(n=sample_size, random_state=42)

# Definimos las características y la variable objetivo
X = df_sample['processed_reviewText']
y = df_sample['overall'].apply(lambda rating: 1 if rating >= 4 else 0)  # 1 para positivo, 0 para negativo

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Creamos el vectorizador Bag-of-Words
vectorizer = CountVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 3)) # ngram_range ya que, como hemos visto, los trigramas también ofrecen información

# Ajustamos y transformamos los datos de entrenamiento
X_train_bow = vectorizer.fit_transform(X_train)

# Transformamos los datos de prueba
X_test_bow = vectorizer.transform(X_test)

# Definimos los parámetros de la búsqueda para Logistic Regression
param_grid_lr = {
    'penalty': ['l2'],  # Limitar a 'l2' para reducir el espacio de búsqueda
    'C': [0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000, 10000],
    'solver': ['liblinear']
}

# Creamos el modelo de Logistic Regression
lr = LogisticRegression(random_state=42)

# Configuramos la búsqueda con GridSearchCV para Logistic Regression
grid_search_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, cv=3, scoring='f1', n_jobs=-1)

# Ajustamos la búsqueda a los datos de entrenamiento
grid_search_lr.fit(X_train_bow, y_train)

# Mejor modelo encontrado por la búsqueda para Logistic Regression
best_lr_model = grid_search_lr.best_estimator_

# Predecimos los valores en el conjunto de prueba con el mejor modelo de Logistic Regression
y_pred_best_lr = best_lr_model.predict(X_test_bow)

# Evaluamos el mejor modelo de Logistic Regression
print(f"Best Logistic Regression Model With C = {grid_search_lr.best_params_['C']}:")
print(classification_report(y_test, y_pred_best_lr))

# Definimos los parámetros de la búsqueda para Random Forest
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Creamos el modelo de Random Forest
rf = RandomForestClassifier(random_state=42)

# Configuramos la búsqueda con GridSearchCV para Random Forest
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=3, scoring='f1', n_jobs=-1)

# Ajustamos la búsqueda a los datos de entrenamiento
grid_search_rf.fit(X_train_bow, y_train)

# Mejor modelo encontrado por la búsqueda para Random Forest
best_rf_model = grid_search_rf.best_estimator_

# Predecimos los valores en el conjunto de prueba con el mejor modelo de Random Forest
y_pred_best_rf = best_rf_model.predict(X_test_bow)

# Evaluamos el mejor modelo de Random Forest
print("Best Random Forest Model:")
print(f"Best parameters: {grid_search_rf.best_params_}")
print(classification_report(y_test, y_pred_best_rf))

# Comparamos las métricas de los mejores modelos
best_lr_report = classification_report(y_test, y_pred_best_lr, output_dict=True)
best_rf_report = classification_report(y_test, y_pred_best_rf, output_dict=True)

# Mostramos las métricas de precisión, recall y f1-score para ambos modelos
print(f"Best Logistic Regression - Precision: {best_lr_report['1']['precision']}, Recall: {best_lr_report['1']['recall']}, F1-score: {best_lr_report['1']['f1-score']}")
print(f"Best Random Forest - Precision: {best_rf_report['1']['precision']}, Recall: {best_rf_report['1']['recall']}, F1-score: {best_rf_report['1']['f1-score']}")


Dataset size after dropping NaN: 49940
Best Logistic Regression Model With C = 0.25:
              precision    recall  f1-score   support

           0       0.71      0.41      0.52      2017
           1       0.87      0.96      0.91      7971

    accuracy                           0.85      9988
   macro avg       0.79      0.68      0.72      9988
weighted avg       0.83      0.85      0.83      9988

Best Random Forest Model:
Best parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
              precision    recall  f1-score   support

           0       0.92      0.06      0.10      2017
           1       0.81      1.00      0.89      7971

    accuracy                           0.81      9988
   macro avg       0.86      0.53      0.50      9988
weighted avg       0.83      0.81      0.73      9988

Best Logistic Regression - Precision: 0.8654195011337869, Recall: 0.9575962865387028, F1-score: 0.9091775355845393
Best Random Forest

Seguimos viendo que la logística es la que mejor funciona  y que no hemos conseguido una mejora significativa al coger los parámetros óptimos. Esto se puede deber a que no estamos realizando una búsqueda intensiva de los mismos ya que el tiempo de carga de los apartados puede crecer de forma exponencial.

Vamos a realizar predicciones con el modelo para visualizar la potencia de estos modelos que  hemos creado.

In [ ]:
import random

def predict_review_sentiment(review_index, model):
    print('Actual sentiment: {}'.format(1 if df.iloc[review_index]['overall'] >= 4 else 0))
    r = df.iloc[review_index]['processed_reviewText']
    print('Prediction: {}'.format(best_lr_model.predict(vectorizer.transform([r]))))

for i in random.sample(range(0, len(df)), 5):
    print('\nReview no. {}'.format(i))
    predict_review_sentiment(i, best_lr_model)


Review no. 38319
Actual sentiment: 1
Prediction: [1]

Review no. 17274
Actual sentiment: 1
Prediction: [1]

Review no. 46942
Actual sentiment: 0
Prediction: [1]

Review no. 42676
Actual sentiment: 1
Prediction: [1]

Review no. 32359
Actual sentiment: 1
Prediction: [1]


In [ ]:
def predict_review_sentiment(review_index, model):
    print('Actual sentiment: {}'.format(1 if df.iloc[review_index]['overall'] >= 4 else 0))
    r = df.iloc[review_index]['processed_reviewText']
    print('Prediction: {}'.format(best_rf_model.predict(vectorizer.transform([r]))))

for i in random.sample(range(0, len(df)), 5):
    print('\nReview no. {}'.format(i))
    predict_review_sentiment(i, best_rf_model)


Review no. 5362
Actual sentiment: 0
Prediction: [1]

Review no. 3726
Actual sentiment: 1
Prediction: [1]

Review no. 5954
Actual sentiment: 1
Prediction: [1]

Review no. 43661
Actual sentiment: 1
Prediction: [1]

Review no. 44486
Actual sentiment: 0
Prediction: [1]


Probemos ahora a usar el TfidfVectorizer en vez del CountVectorizer a ver si conseguimos mejores resultados.

In [9]:
# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Cargamos los datos preprocesados
df = pd.read_csv('processed_pet_supplies_reviews.csv')

# Eliminamos filas con valores NaN en 'processed_reviewText'
df = df.dropna(subset=['processed_reviewText'])

# Verificamos el tamaño del dataset después de eliminar NaN
dataset_size = len(df)
print(f"Dataset size after dropping NaN: {dataset_size}")

# Usamos una muestra del dataset si el tamaño es mayor a 50,000, de lo contrario usamos todo el dataset
sample_size = min(50000, dataset_size)
df_sample = df.sample(n=sample_size, random_state=42)

# Definimos las características y la variable objetivo
X = df_sample['processed_reviewText']
y = df_sample['overall'].apply(lambda rating: 1 if rating >= 4 else 0)  # 1 para positivo, 0 para negativo

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Creamos el vectorizador Bag-of-Words
vectorizer = TfidfVectorizer(
    max_df=0.95,
    min_df=3,
    max_features=2500,
    strip_accents='ascii',
    ngram_range=(1, 3)
)

# Ajustamos y transformamos los datos de entrenamiento
X_train_bow = vectorizer.fit_transform(X_train)

# Transformamos los datos de prueba
X_test_bow = vectorizer.transform(X_test)

# Definimos los parámetros de la búsqueda para Logistic Regression
param_grid_lr = {
    'penalty': ['l2'],  # Limitar a 'l2' para reducir el espacio de búsqueda
    'C': [0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000, 10000],
    'solver': ['liblinear']
}

# Creamos el modelo de Logistic Regression
lr = LogisticRegression(random_state=42)

# Configuramos la búsqueda con GridSearchCV para Logistic Regression
grid_search_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, cv=3, scoring='f1', n_jobs=-1)

# Ajustamos la búsqueda a los datos de entrenamiento
grid_search_lr.fit(X_train_bow, y_train)

# Mejor modelo encontrado por la búsqueda para Logistic Regression
best_lr_model = grid_search_lr.best_estimator_

# Predecimos los valores en el conjunto de prueba con el mejor modelo de Logistic Regression
y_pred_best_lr = best_lr_model.predict(X_test_bow)

# Evaluamos el mejor modelo de Logistic Regression
print(f"Best Logistic Regression Model With C = {grid_search_lr.best_params_['C']}:")
print(classification_report(y_test, y_pred_best_lr))

# Definimos los parámetros de la búsqueda para Random Forest
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Creamos el modelo de Random Forest
rf = RandomForestClassifier(random_state=42)

# Configuramos la búsqueda con GridSearchCV para Random Forest
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=3, scoring='f1', n_jobs=-1)

# Ajustamos la búsqueda a los datos de entrenamiento
grid_search_rf.fit(X_train_bow, y_train)

# Mejor modelo encontrado por la búsqueda para Random Forest
best_rf_model = grid_search_rf.best_estimator_

# Predecimos los valores en el conjunto de prueba con el mejor modelo de Random Forest
y_pred_best_rf = best_rf_model.predict(X_test_bow)

# Evaluamos el mejor modelo de Random Forest
print("Best Random Forest Model:")
print(f"Best parameters: {grid_search_rf.best_params_}")
print(classification_report(y_test, y_pred_best_rf))

# Comparamos las métricas de los mejores modelos
best_lr_report = classification_report(y_test, y_pred_best_lr, output_dict=True)
best_rf_report = classification_report(y_test, y_pred_best_rf, output_dict=True)

# Mostramos las métricas de precisión, recall y f1-score para ambos modelos
print(f"Best Logistic Regression - Precision: {best_lr_report['1']['precision']}, Recall: {best_lr_report['1']['recall']}, F1-score: {best_lr_report['1']['f1-score']}")
print(f"Best Random Forest - Precision: {best_rf_report['1']['precision']}, Recall: {best_rf_report['1']['recall']}, F1-score: {best_rf_report['1']['f1-score']}")


Dataset size after dropping NaN: 49940
Best Logistic Regression Model With C = 1:
              precision    recall  f1-score   support

           0       0.77      0.50      0.61      2017
           1       0.88      0.96      0.92      7971

    accuracy                           0.87      9988
   macro avg       0.83      0.73      0.77      9988
weighted avg       0.86      0.87      0.86      9988

Best Random Forest Model:
Best parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
              precision    recall  f1-score   support

           0       0.95      0.05      0.10      2017
           1       0.81      1.00      0.89      7971

    accuracy                           0.81      9988
   macro avg       0.88      0.52      0.49      9988
weighted avg       0.84      0.81      0.73      9988

Best Logistic Regression - Precision: 0.8847262247838616, Recall: 0.9628653870279764, F1-score: 0.9221434578877808
Best Random Forest - 